This notebook performs named entity recognition to extract all the locations from each article.

Prerequisites:
- Have the `spacy` library installed on your local machine. It would be good to install spaCy with the `lookups` option. You can do this by running `pip install -U spacy[lookups]` in your VSCode terminal. You should install `lookups` if you're training a blank model and you want it to include lemmatization and normalization data.
- Download the NER model you will use to your local machine. To do so, you can run the following command in your VSCode terminal: `python -m spacy download <model_name>`

# Imports

In [1]:
import spacy
import pandas as pd
import re
from spacy import displacy

In [2]:
# This code is needed for the SpaCy NER visualizations to work.
import sys
from IPython import display

# TODO: see if this affects other imports or causes other issues
# `sys.modules` is a dict containing all the modules imported.
# The keys are strings of the module name, the values are the module objects.
sys.modules['IPython.core.display'] = display

# Helper functions

In [3]:
# Include Mark's function for normalizing a piece of location text
def normalize(loc: str) -> str:
    # remove leading/trailing whitespace and punctuation\n",
    loc = (loc or "\\").strip().strip(",.;: ")
    
    # collapse multiple spaces, ensure single space after commas\n",
    loc = re.sub(r"\\s+\\", "\\" "\\", loc)
    # ensure single space after commas\n",
    loc = re.sub(r"\\s*,\\s*\\", "\\, \\", loc)
    return loc

In [4]:
def remove_extra_spaces(text: str) -> str:
    """ 
    Removes multiple consecutive spaces in a text, replacing them with one space.
    Also removes leading and trailing spaces.

    Parameters:
        - text: the text to clean up

    Returns: the text with spaces removed
    """

    text = text.strip() # remove leading/trailing whitespace
    text = re.sub(pattern = r"\\s+", repl = r" ", string = text) # replace multiple spaces w/ a single space
    return text

# Load data

In [5]:
# Read the full dataframe where the data was partially cleaned
df = pd.read_csv("cleaned_police_reports.csv")
df.head()

,name,department,url,text
0,"Andrew Allen, badge #37",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,Home Legislative File 2021-01132 RCA Legal s...
1,"Guled Abdullahi, badge #706",Hennepin County Sheriff's Department,https://assets.nationbuilder.com/cuapb/pages/1...,"Hennepin County 300 South Sixth Street, Minne..."
2,"Dean V. Albers, badge #None",Goodhue County Sheriff's Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"2/22/2021 Jenson v. Craft, Civil No. 01-1488(D..."
3,"Scott Aikins, badge #22",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"2/22/2021 United States v. Diriye, Case No. 14..."
4,"Matthew Aish, badge #None",Columbia Heights Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,Too_Long1 Arbitration LELS (Mathew Aish)/...


In [6]:
# Read the small dataframe of manually cleaned articles
small_df = pd.read_csv("manually_cleaned_police_reports_small.csv")
small_df.head()

,name,department,url,text
0,"Jeffrey Pennaz, badge #5551",Department:Minneapolis Police Department,https://assets.nationbuilder.com/cuapb/pages/1...,Witnesses say the stop happened around 8:30 p....
1,"Kurt Radke, badge #5882",Department:Minneapolis Police Department,https://assets.nationbuilder.com/cuapb/pages/1...,Home Legislative File 2022-00241 RCA Legal S...
2,"Craig A. Taylor, badge #7139",Department:Minneapolis Police Department,https://assets.nationbuilder.com/cuapb/pages/1...,Home Legislative File 2022-00233 RCA Legal S...
3,"Cory Taylor, badge #7141",Department:Minneapolis Police Department,https://assets.nationbuilder.com/cuapb/pages/1...,Home Legislative File 2022-00240 RCA Legal S...
4,"Joseph Will, badge #7749",Department:Minneapolis Police Department,https://assets.nationbuilder.com/cuapb/pages/1...,Home Legislative File 2022-00230 RCA Legal S...


# Try SpaCy's `en_core_web_sm` model for NER

## Try NER on texts in the full dataframe

In [7]:
nlp = spacy.load('en_core_web_sm') # load the NER model

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
for t in df["text"].unique()[:5]:
    print(f"\n{t}")


Home Legislative File 2021-01132 RCA Legal selement: Workers' Compensaon claim of Andrew Allen (RCA-2021-01206) ORIGINATING DEPARTMENT Finance & Property Services To Commiee(s) # Commiee Name Meeng Date 1 Policy & Government Oversight Commiee Oct 20, 2021 LEAD STAFF: Emily Ann Colby PRESENTED BY: Emily Ann Colby Acon Item(s) # File Type Subcategory Item Descripon 1 Acon Selement Approving the selement of the Workers' Compensaon claim of Andrew Allen, by payment of $170,000 to Andrew Allen and aorney, Meuser Law Firm, and authorizing the City Aorney's Oﬃce to execute any documents necessary to eﬀectuate the selement. Previous Acons None RCA-2021-01206 - Legal settlement: Workers' Compensation claim of ... https://lims.minneapolismn.gov/RCA/8755 1 of 2 12/4/2021, 1:33 AM Ward / Neighborhood / Address # Ward Neighborhood Address 1. Not Applicable Background Analysis City of Minneapolis employee sustained work-related injuries. The pares reached a tentave selement of 

We will start off by trying the 5th text that was printed. 

In [ ]:
text = df.loc[df["text"].str.startswith("Too_Long1    Arbitration   LELS"), "text"].values[0] # get the text in that row
print(f"Text before cleaning:\n{text}")
cleaned_text = remove_extra_spaces(text) # clean it

Text before cleaning:
Too_Long1    Arbitration   LELS (Mathew Aish)/Columbia Heights Police Department    Grievance Arbitration Between:        BMS Case No.: 20 PA-1575    Columbia Heights Police Dept. (Employer)   and   Law Enforcement Labor Services Inc. (LELS)  Mathew Aish (Grievant)     REPRESENTATION  Representing Columbia Heights Police Department  Joan Quade, Attorney   BARNA, GUZY & STEFFEN, LTD.    Representing Mathew Aish:  Renee Zachman, Attorney  Mark Schneider, Attorney  Law Enforcement Labor Services  ISSUE  Did the employer have Just Cause to Terminate Mathew Aish?  If not, what is the appropriate Remedy?    INTRODUCTION  Mathew Aish was a police officer in Columbia Heights hired in 1999 and terminated on January 15, 2020.  Law  Enforcement Labor Services responded to the termination by filing a step one grievance on January 27, 2020.  The termination stems primarily from an incident that occurred on October 27, 2019 in which the grievant,  Mathew Aish, responded to a di

In [ ]:
print(f"Text after cleaning:\n{cleaned_text}")

Text after cleaning:
Too_Long1    Arbitration   LELS (Mathew Aish)/Columbia Heights Police Department    Grievance Arbitration Between:        BMS Case No.: 20 PA-1575    Columbia Heights Police Dept. (Employer)   and   Law Enforcement Labor Services Inc. (LELS)  Mathew Aish (Grievant)     REPRESENTATION  Representing Columbia Heights Police Department  Joan Quade, Attorney   BARNA, GUZY & STEFFEN, LTD.    Representing Mathew Aish:  Renee Zachman, Attorney  Mark Schneider, Attorney  Law Enforcement Labor Services  ISSUE  Did the employer have Just Cause to Terminate Mathew Aish?  If not, what is the appropriate Remedy?    INTRODUCTION  Mathew Aish was a police officer in Columbia Heights hired in 1999 and terminated on January 15, 2020.  Law  Enforcement Labor Services responded to the termination by filing a step one grievance on January 27, 2020.  The termination stems primarily from an incident that occurred on October 27, 2019 in which the grievant,  Mathew Aish, responded to a dis

In [ ]:
text == cleaned_text # BUG: text cleaning function doesn't change the text

True

In [ ]:
doc = nlp(text) # Process the input text and tokenizes it. We can get the entities from the resulting `Doc` object

# Go through all the predicted named entities in the text
for ent in doc.ents:
    print(f"\n{ent.text} | {ent.start_char} | {ent.end_char} | {ent.label_}")


Mathew Aish)/Columbia Heights Police Department    Grievance Arbitration | 33 | 105 | ORG

20 | 136 | 138 | CARDINAL

PA-1575     | 139 | 150 | LOC

Law Enforcement Labor Services Inc. | 199 | 234 | ORG

Mathew Aish | 243 | 254 | PERSON

Joan Quade | 335 | 345 | PERSON

GUZY & STEFFEN | 365 | 379 | ORG

LTD | 381 | 384 | ORG

Renee Zachman | 416 | 429 | PERSON

Mark Schneider | 441 | 455 | PERSON

Law Enforcement Labor Services | 467 | 497 | ORG

Terminate Mathew Aish | 542 | 563 | PERSON

Remedy | 598 | 604 | PERSON

Mathew Aish | 623 | 634 | PERSON

Columbia Heights | 659 | 675 | GPE

1999 | 685 | 689 | DATE

January 15, 2020 | 708 | 724 | DATE

Law  Enforcement Labor Services | 727 | 758 | ORG

one | 805 | 808 | CARDINAL

January 27, 2020 | 822 | 838 | DATE

October 27, 2019 | 907 | 923 | DATE

Mathew Aish | 948 | 959 | PERSON

2 | 1060 | 1061 | CARDINAL

Fridley | 1086 | 1093 | PERSON

Taurus | 1151 | 1157 | PRODUCT

80 miles | 1183 | 1191 | QUANTITY

Mathew Aish | 1387 | 1398 | P

In [ ]:
# Get a string description of what each of these entity type labels means

for label in ["LOC", "ORG", "CARDINAL", "GPE", "DATE", "PRODUCT", "QUANTITY", "PERSON", "TIME", "FAC", "NORP"]:
    print(f"{label}: {spacy.explain(label)}\n")

LOC: Non-GPE locations, mountain ranges, bodies of water

ORG: Companies, agencies, institutions, etc.

CARDINAL: Numerals that do not fall under another type

GPE: Countries, cities, states

DATE: Absolute or relative dates or periods

PRODUCT: Objects, vehicles, foods, etc. (not services)

QUANTITY: Measurements, as of weight or distance

PERSON: People, including fictional

TIME: Times smaller than a day

FAC: Buildings, airports, highways, bridges, etc.

NORP: Nationalities or religious or political groups



In [ ]:
# Visualize the entities and their labels by highlighting them in the text. Render the visualization as an HTML markup, which we can later save as an image
displacy.render(doc, style="ent", jupyter=True)

This NER output is very wrong. For example, `"Mathew Aish)/Columbia Heights Police Department    Grievance Arbitration"` is not an organization, and "Columbia Heights" is one location, not two separate locations.

Now look for articles in the dataframe rather than court cases.

In [ ]:
df.loc[df["text"].str.contains("News")]

,name,department,url,text
2,"Dean V. Albers, badge #None",Goodhue County Sheriff's Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"2/22/2021 Jenson v. Craft, Civil No. 01-1488(D..."
3,"Scott Aikins, badge #22",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"2/22/2021 United States v. Diriye, Case No. 14..."
5,"Vincent A. Adams, badge #720505",St. Paul Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,3/13/2021 Prosecutors: St. Paul cops justified...
9,"Roxanne Affeldt, badge #None",Champlin Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"PRISONLEGALNEWS.ORG › Annotations $೔ೌ,ೌೌೌ M..."
11,"Brandon Akers, badge #None",Brooklyn Center Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,4/12/2021 Officers ID'd in Brooklyn Center fat...
...,...,...,...,...
2220,"Joshua James Williams, badge #None",Dakota County Sheriff's Office,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,"Deputy gets license suspension, 40 hours comm..."
2238,"Jason Wolff, badge #7859",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,4/12/2021 News Releases - Officers Identified ...
2242,"Aaron Womble, badge #7851",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,4/12/2021 News Releases - Officers Identified ...
2264,"Toua Yang, badge #7892",Minneapolis Police Department,https://d3n8a8pro7vhmx.cloudfront.net/cuapb/pa...,4/12/2021 News Releases - Officers Identified ...


Examine the rows that are of interest:

In [ ]:
text = df.loc[9, "text"] # this article has a lot of weird characters, so let's see if the NER model can handle those
text

'PRISONLEGALNEWS.ORG ›  Annotations   $\u0cd4ೌ,ೌೌೌ Minnesota Illegal Search & Seizure Verdict Upheld; Attorney Fees Slashed \u0cd1ೌ% MARK WILSON AUGUST ೄೄ, \u0cc5ೃೄ\u0cc9 Share on Twitter Share on Facebook Share on G+ Share with email A Minnesota federal court upheld an \uf653\uf64b\uf643,\uf643\uf643\uf643 jury verdict on illegal search and seizure claims. \ue053e court reduced Plaintiﬀs\' requested attorney\'s fees by \uf648\uf643 percent, for a total fee award of \uf653\uf644\uf645\uf64b,\uf647\uf64c\uf64c.\uf64b\uf648. Just before \uf644 a.m., on January \uf64b, \uf645\uf643\uf644\uf643, Champlin, Minnesota police responded to a veterinary hospital burglary. Police used a canine to track the suspect who ﬂed the crime scene. \ue053e canine lost the scent but eventually led Sergeant Bill Schmidt and Oﬃcers Roxanne Aﬀeldt and Robert Topp, to the home of Ronald Rosen and June Trnka. Aﬀeldt knocked on the front door but received no response. Schmidt and Aﬀeldt then went to the rear of t

In [ ]:
doc = nlp(text)

for ent in doc.ents:
    print(f"\n{ent.text} | {ent.start_char} | {ent.end_char} | {ent.label_}")


›  Annotations | 20 | 34 | PERSON

Minnesota Illegal Search & Seizure Verdict Upheld | 45 | 94 | ORG

AUGUST | 134 | 140 | DATE

೅ೃೄ೉ Share | 145 | 155 | PERSON

Twitter Share on Facebook Share on G+ Share | 159 | 202 | WORK_OF_ART

Minnesota | 216 | 225 | GPE

, | 250 | 257 | ORG

Plaintiﬀs | 326 | 335 | ORG

 percent | 366 | 376 | GPE

January | 439 | 446 | DATE

 | 450 | 454 | ORG

Champlin | 456 | 464 | ORG

Minnesota | 466 | 475 | GPE

Bill Schmidt | 648 | 660 | PERSON

Roxanne Aﬀeldt | 672 | 686 | PERSON

Robert Topp | 691 | 702 | PERSON

Ronald Rosen | 719 | 731 | PERSON

June | 736 | 740 | DATE

Aﬀeldt | 748 | 754 | NORP

Schmidt | 807 | 814 | PERSON

Aﬀeldt | 819 | 825 | PERSON

Rosen | 1017 | 1022 | PERSON

ey | 1032 | 1035 | PERSON

Rosen | 1262 | 1267 | PERSON

fourteen ✕ days later | 1362 | 1383 | DATE

January | 1388 | 1395 | DATE

Champlin | 1406 | 1414 | NORP

Rosen | 1431 | 1436 | PERSON

more than an hour | 1441 | 1458 | TIME

Brian Wentworth | 1498 | 15

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

This is still very wrong! For instance, "Rosen v. Wentworth" is not a person, and the "June" in "June Trnka" is the name of a person, not a date. The weird characters are also obscuring important information, like the date.

In [ ]:
from ftfy import fix_text
fix_text(text)

'PRISONLEGALNEWS.ORG ›  Annotations   $\u0cd4ೌ,ೌೌೌ Minnesota Illegal Search & Seizure Verdict Upheld; Attorney Fees Slashed \u0cd1ೌ% MARK WILSON AUGUST ೄೄ, \u0cc5ೃೄ\u0cc9 Share on Twitter Share on Facebook Share on G+ Share with email A Minnesota federal court upheld an \uf653\uf64b\uf643,\uf643\uf643\uf643 jury verdict on illegal search and seizure claims. \ue053e court reduced Plaintiffs\' requested attorney\'s fees by \uf648\uf643 percent, for a total fee award of \uf653\uf644\uf645\uf64b,\uf647\uf64c\uf64c.\uf64b\uf648. Just before \uf644 a.m., on January \uf64b, \uf645\uf643\uf644\uf643, Champlin, Minnesota police responded to a veterinary hospital burglary. Police used a canine to track the suspect who fled the crime scene. \ue053e canine lost the scent but eventually led Sergeant Bill Schmidt and Officers Roxanne Affeldt and Robert Topp, to the home of Ronald Rosen and June Trnka. Affeldt knocked on the front door but received no response. Schmidt and Affeldt then went to the re

Some of the "\uxxxx" characters in the article above appear to be Unicode characters in different languages, or invalid Unicode characters.

## Try NER on the small dataframe

In [ ]:
for t in small_df["text"]:
    print(f"\n{t}")


Witnesses say the stop happened around 8:30 p.m. outside 1515 Nicollet Ave, Minneapolis, MN. The complainant alleges the officer pulled them from the vehicle without cause and placed them in cuffs on the sidewalk."

Home Legislative File 2022-00241 RCA Legal Selement: Workers' Compensaon claim of Kurt Radke (RCA-2022-00194) ORIGINATING DEPARTMENT Finance & Property Services To Commiee(s) # Commiee Name Meeng Date 1 Policy & Government Oversight Commiee Mar 7, 2022 LEAD STAFF: Emily Ann Colby PRESENTED BY: Emily Ann Colby Acon Item(s) # File Type Subcategory Item Descripon 1 Acon Selement Approving the workers' compensaon claim of Kurt Radke by payment of $160,000 over three years to Kurt Radke and aorney, Meuser Law Firm, and authorizing the City Aorney's Oﬃce to execute any documents necessary to eﬀectuate the selement. Ward / Neighborhood / Address # Ward Neighborhood Address 1. Not Applicable Background Analysis City of Minneapolis employee sustained work-related in

We will try the first and last texts in `small_df`.

In [ ]:
text = small_df.loc[len(small_df) - 1, "text"] # last text
text

'According to the complaint, the incident began at the intersection of W 7th St & Kellogg Blvd, St. Paul, MN. Bystanders reported raised voices and a brief scuffle before the officer escorted the resident to a patrol car.'

In [ ]:
doc = nlp(text)

for ent in doc.ents:
    print(f"\n{ent.text} | {ent.start_char} | {ent.end_char} | {ent.label_}")


W 7th St & Kellogg Blvd | 70 | 93 | ORG

St. Paul | 95 | 103 | GPE

MN | 105 | 107 | ORG


This is completely wrong too because "W 7th St & Kellogg Blvd" is not an organization, and W 7th St & Kellogg Blvd, St. Paul, MN is supposed to be one location.

In [ ]:
text = small_df.loc[0, "text"] # first text
text

'Witnesses say the stop happened around 8:30 p.m. outside 1515 Nicollet Ave, Minneapolis, MN. The complainant alleges the officer pulled them from the vehicle without cause and placed them in cuffs on the sidewalk."'

In [ ]:
doc = nlp(text)

for ent in doc.ents:
    print(f"\n{ent.text} | {ent.start_char} | {ent.end_char} | {ent.label_}")


around 8:30 p.m. | 32 | 48 | TIME

1515 | 57 | 61 | DATE

Nicollet Ave | 62 | 74 | ORG

Minneapolis | 76 | 87 | GPE

MN | 89 | 91 | ORG


This output is completely wrong too.

In [ ]:
# Try a simple sentence
text = "Jane Smith lives on 123 Redmond Ave. NE, Redmond, WA, USA with two dogs, in a red house. She works for Microsoft."
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

The model cannot correctly extract locations at all.

# Try SpaCy's `en_core_web_trf`

(Transformer trained on web data).

## Try small dataframe

In [ ]:
transformer = spacy.load('en_core_web_trf') # takes 52 sec to load

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torc

In [ ]:
text = small_df.loc[0, "text"] # first text
text


'Witnesses say the stop happened around 8:30 p.m. outside 1515 Nicollet Ave, Minneapolis, MN. The complainant alleges the officer pulled them from the vehicle without cause and placed them in cuffs on the sidewalk."'

In [ ]:
doc = transformer(text)
displacy.render(doc, style="ent", jupyter=True)

Doesn't detect the street address, and it detects "Minneapolis" and "MN" as two separate locations, not one.

In [ ]:
text = small_df.loc[len(small_df) - 1, "text"] # last text
text

'According to the complaint, the incident began at the intersection of W 7th St & Kellogg Blvd, St. Paul, MN. Bystanders reported raised voices and a brief scuffle before the officer escorted the resident to a patrol car.'

In [ ]:
doc = transformer(text)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
for label in ["FAC", "GPE"]:
    print(f"{label}: {spacy.explain(label)}")

FAC: Buildings, airports, highways, bridges, etc.
GPE: Countries, cities, states


This one is slightly better. It detects "Kellogg Blvd", but it does not detect the full intersection name (W 7th St & Kellogg Blvd). It also detects the street, city, and state as separate locations, not one location.

In [ ]:
text = small_df.loc[1, "text"] # 2nd text
text

"Home \uf105Legislative File 2022-00241 \uf105RCA Legal Se\x01lement: Workers' Compensa\x02on claim of Kurt Radke (RCA-2022-00194) ORIGINATING DEPARTMENT Finance & Property Services To Commi\x01ee(s) # Commi\x01ee Name Mee\x02ng Date 1 Policy & Government Oversight Commi\x01ee Mar 7, 2022 LEAD STAFF: Emily Ann Colby PRESENTED BY: Emily Ann Colby Ac\x02on Item(s) # File Type Subcategory Item Descrip\x02on 1 Ac\x02on Se\x01lement Approving the workers' compensa\x02on claim of Kurt Radke by payment of $160,000 over three years to Kurt Radke and a\x01orney, Meuser Law Firm, and authorizing the City A\x01orney's Oﬃce to execute any documents necessary to eﬀectuate the se\x01lement. Ward / Neighborhood / Address # Ward Neighborhood Address 1. Not Applicable Background Analysis City of Minneapolis employee sustained work-related injuries. The par\x02es reached a tenta\x02ve se\x01lement by payment of $160,000 over three years from fun 06930-1450100-789401-145400. Risk Management believes this

In [ ]:
doc = transformer(text)
displacy.render(doc, style="ent", jupyter=True)

The model incorrectly detects "Action" (with the weird character) as part of a person's name. Also, "Ward" is not a person, and "the Committee and the" is not a full organization name. The model is fairly good at detecting person names.

In [ ]:
# Try a simple sentence
text = "Jane Smith lives on 123 Redmond Ave. NE, Redmond, WA, USA with two dogs, Roger and Fido, in a red house. She works for Microsoft."
doc = transformer(text)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
text = "Jane Smith lives at the address 123 Redmond Avenue NE, Redmond, WA, USA with two dogs, Roger and Fido, in a red house. She works for Microsoft."
doc = transformer(text)
displacy.render(doc, style="ent", jupyter=True)

Still can't detect a street address as a location, and it still detects city/state/country as separate locations.

## Try full dataframe

In [ ]:
for t in df["text"][:5]:
    print(f"\n{t}")


Home Legislative File 2021-01132 RCA Legal selement: Workers' Compensaon claim of Andrew Allen (RCA-2021-01206) ORIGINATING DEPARTMENT Finance & Property Services To Commiee(s) # Commiee Name Meeng Date 1 Policy & Government Oversight Commiee Oct 20, 2021 LEAD STAFF: Emily Ann Colby PRESENTED BY: Emily Ann Colby Acon Item(s) # File Type Subcategory Item Descripon 1 Acon Selement Approving the selement of the Workers' Compensaon claim of Andrew Allen, by payment of $170,000 to Andrew Allen and aorney, Meuser Law Firm, and authorizing the City Aorney's Oﬃce to execute any documents necessary to eﬀectuate the selement. Previous Acons None RCA-2021-01206 - Legal settlement: Workers' Compensation claim of ... https://lims.minneapolismn.gov/RCA/8755 1 of 2 12/4/2021, 1:33 AM Ward / Neighborhood / Address # Ward Neighborhood Address 1. Not Applicable Background Analysis City of Minneapolis employee sustained work-related injuries. The pares reached a tentave selement of 

In [ ]:
# Try the second and fifth texts. The lambda function truncates the text to only the first 3000 characters, for brevity.
texts = df.loc[[1, 4], "text"].apply(lambda s: s if len(s) < 3000 else s[:3000]).values

for text in texts:
    text = remove_extra_spaces(text)
    print(f"Text:\n{text}\n\nVisualization:")

    doc = transformer(text)
    displacy.render(doc, style="ent", jupyter=True)

Text:
Hennepin County  300 South Sixth Street, Minneapolis, MN 55487    DATE:   December 21, 2021      TO:     Detention Deputy Guled Abdullahi      FROM:   Assistant County Administrator Mark Thompson      RE:     Written Reprimand         You are receiving a written reprimand for your failure to follow Hennepin  County’s COVID-19 Vaccination and Testing Policy.  Hennepin County has sent regular  and ongoing communication regarding the policy expectation of being fully vaccinated  or complying with weekly COVID testing.        During the week of October 11-16, 2021 you failed to provide proof:     •  Of your vaccination status, or   •  That you had taken a COVID test and provided your test results      Due to the seriousness of your failure to follow directives and abide by the county’s  vaccination and testing policy, you are being issued this written reprimand.        It is my hope that you will take advantage of this opportunity to correct your behavior so  that you may succeed in 

Text:
Too_Long1    Arbitration   LELS (Mathew Aish)/Columbia Heights Police Department    Grievance Arbitration Between:        BMS Case No.: 20 PA-1575    Columbia Heights Police Dept. (Employer)   and   Law Enforcement Labor Services Inc. (LELS)  Mathew Aish (Grievant)     REPRESENTATION  Representing Columbia Heights Police Department  Joan Quade, Attorney   BARNA, GUZY & STEFFEN, LTD.    Representing Mathew Aish:  Renee Zachman, Attorney  Mark Schneider, Attorney  Law Enforcement Labor Services  ISSUE  Did the employer have Just Cause to Terminate Mathew Aish?  If not, what is the appropriate Remedy?    INTRODUCTION  Mathew Aish was a police officer in Columbia Heights hired in 1999 and terminated on January 15, 2020.  Law  Enforcement Labor Services responded to the termination by filing a step one grievance on January 27, 2020.  The termination stems primarily from an incident that occurred on October 27, 2019 in which the grievant,  Mathew Aish, responded to a dispatch call rega

Maybe when the model detects "Hennepin" and "County" as separate entities, that's because of the extra spaces between characters?

In [ ]:
remove_extra_spaces()